<a href="https://colab.research.google.com/github/Julioore/TFM/blob/main/TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valoración de opciones financieras
* Método Binomial
* Método de Black-Scholes
* Convergencia entre ambos métodos
* Gráfico de convergencia

In [ ]:
import numpy as np                              # importamos las librerías
from scipy.stats import binom
from scipy.stats import norm
import matplotlib as mpl
import matplotlib.pyplot as plt

# Fórmula para método de valoración opciones europeas mediante árbol binomial

def prima(tipo, spot, strik, anyos, volatilidad, n, tInt):
    r = np.log(1+tInt)                          # r es el tanto instantáneo
    deltaT = anyos / n
    u = np.exp(volatilidad * np.sqrt(deltaT))
    d = 1/u
    p = (np.exp(r*deltaT)-d) / (u-d)
    Sn = [0]*(n+1)                              # Sn es la lista de precios del activo subyacene a fecha de vto.
    payoff = [0]*(n+1)                          # inicilizamos a cero la lista del payoff
    prima = 0
    for i in range(n+1):
        Sn[i] = spot * u**(n-i) * d**i
        if tipo.lower() == 'call':
            payoff[i] = max(0, Sn[i] - strik)
        if tipo.lower() == 'put':
            payoff[i] = max(0, strik - Sn[i])
        prima += payoff[i] * binom.pmf(n-i, n, p)
    prima *= np.exp(-r * anyos)                 # descontamos a valor actual, hasta t=0
    return prima

# Fórmula para método de Black-Scholes

def primaBS(tipo, spot, strik, anyos, volatilidad, tInt):
    r = np.log(1+tInt)                          # r es el tanto instantáneo
    d1 = (np.log(spot/strik) + (r + 0.5 * volatilidad**2) * anyos) / (volatilidad * np.sqrt(anyos))
    d2 = d1 - volatilidad * np.sqrt(anyos)
    if tipo.lower() == 'call':
      primaBS = spot * norm.cdf(d1) - strik * np.exp(-r * anyos) * norm.cdf(d2)
    if tipo.lower() == 'put':
      primaBS = strik * np.exp(-r * anyos) * norm.cdf(-d2) - spot * norm.cdf(-d1)
    return primaBS

# Representamos gráficamente la valoración por el método binomial
# Cuando el número de pasos del método binomial tiende a infinito el 
# valor teórico obtenido coincide con el que proporciona el método de Black-Scholes

m = 200                                   # Calcularemos el arbol desde 1 hasta m pasos
evolucion = [0]*(m)
for j in range(m):
    evolucion[j] = prima('call', 100, 110, 2, 0.2, j+1, 0.07)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
last = 1
bound = m
ax.plot(evolucion)
fig.savefig('graph.png')
print('Prima(1)=', evolucion[0])
print('Prima(2)=', evolucion[1])
print('Prima(3)=', evolucion[2])
print('Prima(4)=', evolucion[3])
print('Prima('+str(m)+')=', evolucion[m-1])
print()
BS = primaBS('call', 100, 110, 2, 0.2, 0.07)
print('Prima Black-Scholes=', BS)
print()

In [ ]:
# Analizamos la convergencia entre ambos métodos
# y representamos el gráfico de convergencia
# Se comprueba que cuando el número de pasos del método binomial tiende a infinito 
# la diferencia entre el valor teórico obtenido y el valor que proporciona el 
# método de Black-Scholes tiende a 0.


m = 200                                   
diferencia = [0]*(m)
for j in range(m):
    diferencia[j] = evolucion[j] - BS
   # print(j+1, evolucion[j], BS, diferencia[j])


fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
last = 1
bound = m
ax.plot(diferencia)
fig.savefig('graph.png')

# Diseño de estrategias con opciones financieras



In [45]:
# Datos de entrada para la opción A
numA = float(input('Número de contratos opción A: ') or 1)    # número de contratos de la opción A
print('Número opciones A=', numA)                                                      
operacionA = input('Operación con opción A: Compra o Venta. ') or 'Compra'   # tipo de operación: compra o venta
if operacionA.lower() == 'compra':
    print('Tipo de operación de la opción A: Compra')
else:
    print('Tipo de operación de la opción A: Venta')                                                    
tipoA = input('Tipo de opción: Call o Put. ') or 'Call'       # tipo de opción: CALL o PUT
if tipoA.lower() == 'call':
    print('Tipo de la opción A: CALL')
else:
    print('Tipo de la opción A: PUT')                     
strikA = float(input('Precio de ejercicio opción A: ') or 90) # precio de ejercicio de la opción A
print('Precio de ejercicio opción A=', strikA)                                                        
primaA = float(input('Prima de la opción A: ') or 8)          # importe de la prima de la opción A
print('Prima de la opción A=', primaA)
print()                                                        

# Datos de entrada para la opción B                           # idem para opción B
numB = float(input('Número de contratos opción B: ') or 1)    
print('Número opciones B=', numB)                                                          
operacionB = input('Operación con opción B: Compra o Venta. ') or 'Compra'   
if operacionB.lower() == 'compra':
    print('Tipo de operación de la opción B: Compra')
else:
    print('Tipo de operación de la opción B: Venta') 
tipoB = input('Tipo de opción: Call o Put. ') or 'Call'
if tipoB.lower() == 'call':
    print('Tipo de la opción B: CALL')
else:
    print('Tipo de la opción B: PUT')
strikB = float(input('Precio de ejercicio opción B: ') or 100)  
print('Precio de ejercicio opción B=', strikB) 
primaB = float(input('Prima de la opción B: ') or 6)          
print('Prima de la opción B=', primaB)
print()

# Datos de entrada para la opción C                            # idem para opción C
numC = float(input('Número de contratos opción C: ') or 1)    
print('Número opciones C=', numC)                                                                     
operacionC = input('Operación con opción C: Compra o Venta. ') or 'Compra'   
if operacionC.lower() == 'compra':
    print('Tipo de operación de la opción C: Compra')
else:
    print('Tipo de operación de la opción C: Venta') 
tipoC = input('Tipo de opción: Call o Put. ') or 'Call'
if tipoC.lower() == 'call':
    print('Tipo de la opción C: CALL')
else:
    print('Tipo de la opción C: PUT')
strikC = float(input('Precio de ejercicio opción C: ') or 120)  
print('Precio de ejercicio opción C=', strikC) 
primaC = float(input('Prima de la opción C: ') or 6)          
print('Prima de la opción C=', primaC)
print()

# Datos de entrada para la opción D                            # idem para opción D
numD = float(input('Número de contratos opción D: ') or 1)    
print('Número opciones D=', numD)                              
operacionD = input('Operación con opción D: Compra o Venta. ') or 'Compra'   
if operacionD.lower() == 'compra':
    print('Tipo de operación de la opción D: Compra')
else:
    print('Tipo de operación de la opción D: Venta') 
tipoD = input('Tipo de opción: Call o Put. ') or 'Call'
if tipoD.lower() == 'call':
    print('Tipo de la opción D: CALL')
else:
    print('Tipo de la opción D: PUT')
strikD = float(input('Precio de ejercicio opción D: ') or 130)  
print('Precio de ejercicio opción D=', strikD) 
primaD = float(input('Prima de la opción D: ') or 8)          
print('Prima de la opción D=', primaD)
print()

# Evolución de precios del activo subyacente
start=80
stop=141

subyacente = list(range(start,stop))
# print(subyacente)

# Evolución precios subyacente para opción A
subyacenteA=[0]*(stop-start)
if tipoA.lower()=='call':
  for i in range(stop-start):
    subyacenteA[i] = subyacente[i]
else:
  for i in range(stop-start):
    subyacenteA[i] = strikA - subyacente[i] + strikA
# print(subyacenteA)

# Evolución precios subyacente para opción B
subyacenteB=[0]*(stop-start)
if tipoB.lower()=='call':
  for i in range(stop-start):
    subyacenteB[i] = subyacente[i]
else:
  for i in range(stop-start):
    subyacenteB[i] = strikB - subyacente[i] + strikB
# print(subyacenteB)

# Evolución precios subyacente para opción C
subyacenteC=[0]*(stop-start)
if tipoC.lower()=='call':
  for i in range(stop-start):
    subyacenteC[i] = subyacente[i]
else:
  for i in range(stop-start):
    subyacenteC[i] = strikC - subyacente[i] + strikC
# print(subyacenteC)

# Evolución precios subyacente para opción D
subyacenteD=[0]*(stop-start)
if tipoD.lower()=='call':
  for i in range(stop-start):
    subyacenteD[i] = subyacente[i]
else:
  for i in range(stop-start):
    subyacenteD[i] = strikD - subyacente[i] + strikD
# print(subyacenteD)



Número de contratos opción A: 
Número opciones A= 1.0
Operación con opción A: Compra o Venta. 
Tipo de operación de la opción A: Compra
Tipo de opción: Call o Put. 
Tipo de la opción A: CALL
Precio de ejercicio opción A: 
Precio de ejercicio opción A= 90.0
Prima de la opción A: 
Prima de la opción A= 8.0

Número de contratos opción B: 
Número opciones B= 1.0
Operación con opción B: Compra o Venta. 
Tipo de operación de la opción B: Compra
Tipo de opción: Call o Put. 
Tipo de la opción B: CALL
Precio de ejercicio opción B: 
Precio de ejercicio opción B= 100.0
Prima de la opción B: 
Prima de la opción B= 6.0

Número de contratos opción C: 
Número opciones C= 1.0
Operación con opción C: Compra o Venta. 
Tipo de operación de la opción C: Compra
Tipo de opción: Call o Put. 
Tipo de la opción C: CALL
Precio de ejercicio opción C: 
Precio de ejercicio opción C= 120.0
Prima de la opción C: 
Prima de la opción C= 6.0

Número de contratos opción D: 
Número opciones D= 1.0
Operación con opción D:

In [27]:
strikA=90
start=80
stop=141
subyacente = list(range(start,stop))
print(subyacente)
subyacenteA=[0]*(stop-start)
for i in range(stop-start):
  subyacenteA[i] = strikA - subyacente[i] + strikA
print(subyacenteA)

[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140]
[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40]


In [29]:
tipoA='put'
strikA=90
# Evolución de precios del activo subyacente
start=80
stop=141

subyacente = list(range(start,stop))
print(subyacente)

subyacenteA=[0]*(stop-start)
if tipoA.lower()=='call':
  for i in range(stop-start):
    subyacenteA[i] = subyacente[i]
else:
  for i in range(stop-start):
    subyacenteA[i] = strikA - subyacente[i] + strikA
print(subyacenteA)

[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140]
[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40]


In [42]:
tipo = input('Tipo de opción: Call o Put. ') or 'Call'
if tipo.lower() == 'call':
    print('Tipo de la opción: CALL')
else:
    print('Tipo de la opción: PUT')

Tipo de opción: Call o Put. 
Tipo de la opción: CALL
